# Text Clustering and Analysis Pipeline

This notebook walks through a pipeline for clustering text data, specifically customer support training data. The process involves several key stages:

1.  **Data Loading**: Load a dataset of customer support interactions from a CSV file.
2.  **Feature Extraction**: Convert text into numerical representations (embeddings) and then reduce the dimensionality of these embeddings to make them more suitable for clustering.
3.  **Instruction Generation**: Create a set of instructions from the sample data.
4.  **Clustering**: Group the generated instructions into clusters based on their semantic similarity.
5.  **Cluster Description & Matching**: Generate a descriptive title for each cluster and match it back to the original sample categories.
6.  **Evaluation**: Assess the quality of the clustering using a confusion matrix and other similarity scores.

Each step is contained in its own cell, with explanations of the code and guidance on how to modify it.

## 1. Imports and Setup

First, we import all the necessary libraries and modules. This includes utilities for data handling, logging, machine learning models, and our custom pipeline functions.

In [ ]:
# Preloading env vars, seeds and models
import os
from pathlib import Path
import functools
import time
from os import PathLike

import torch
from loguru import logger
from pycm import ConfusionMatrix

from qcluster import tqdm
from tqdm import tqdm
from qcluster import ROOT_DIR
from qcluster.preload import MODEL

# Clustering algorithms
from qcluster.algorithms.clustering import (
    kmeans_clustering,
    # dbscan_clustering,
    # hdbscan_clustering,
    # agglomerative_clustering,
    # bert_topic_extraction,
    # spectral_clustering
)

# Feature extractors and dimensionality reduction
from qcluster.algorithms.feature_extractors import (
    create_embeddings,
    # pca_reduction,
    umap_reduction,
    # pacmap_reduction
)

# Data models and custom types
from qcluster.custom_types import CategoryType, IdToCategoryResultType
from qcluster.datamodels.instruction import InstructionCollection
from qcluster.datamodels.sample import SampleCollection

# Other pipeline components
from qcluster.llm.describer import get_description
from qcluster.evaluation import evaluate_results, cluster_to_class_similarity_measures
from qcluster.algorithms.similarity import get_top_n_similar_embeddings

## 2. Configuration and Hyperparameters

This section defines the core components of our pipeline and their parameters. By modifying the variables in this cell, you can easily experiment with different algorithms and settings.

### How to Modify:
-   **`clustering_function`**: To change the clustering algorithm, comment out `kmeans_clustering` and uncomment another option like `hdbscan_clustering`. Adjust the parameters accordingly. For instance, K-Means requires `n_clusters`, while HDBSCAN uses `min_cluster_size`.
-   **`feature_extractor`**: To change the dimensionality reduction technique, replace `umap_reduction` with another imported function like `pca_reduction` or `pacmap_reduction`. You can also adjust the `n_components` parameter, which determines the final number of dimensions for the embeddings.
-   **`similarity_function`**: You can toggle `use_mmr` (Maximal Marginal Relevance) to control the diversity of similarity search results. When `use_mmr=True`, you can also tune the `mmr_lambda` parameter.

In [ ]:
# Path to the dataset
CSV_PATH: PathLike = (
        ROOT_DIR.parent
        / "data"
        / "Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"
)

# Dynamically determine the number of categories from the data
N_CATEGORIES = len(SampleCollection.all_category_classes())
logger.info(f"Found {N_CATEGORIES} unique categories.")

# --- Clustering Algorithm Configuration ---
clustering_function = functools.partial(
    kmeans_clustering,
    n_clusters=N_CATEGORIES
    # --- Alternative: HDBSCAN ---
    # hdbscan_clustering,
    # min_cluster_size=15, # Minimum size for a group to be considered a cluster
)

# --- Cluster Describer Configuration ---
describer = functools.partial(
    get_description,
    template_name='description_prompt_simple',
    # --- Alternative: More detailed template ---
    # template_name='description_prompt_from_instructions'
)

# --- Similarity Function Configuration ---
similarity_function = functools.partial(
    get_top_n_similar_embeddings,
    use_mmr=False,
    # --- MMR Parameters (for diversifying results) ---
    # mmr_lambda=0.3, # Set between 0 (focus on similarity) and 1 (focus on diversity)
    # mmr_top_n=20
)

# --- Feature Extraction Pipeline ---
def feature_extractor(texts: list[str]) -> torch.Tensor:
    """
    Creates embeddings for the given texts and reduces their dimensionality.
    """
    # Step 1: Create high-dimensional embeddings from text
    embeddings = create_embeddings(texts, model=MODEL)
    
    # Step 2: Reduce dimensionality
    # Recommended to use a value between 5 and 50 for n_components
    reduced_embeddings = umap_reduction(embeddings, n_components=28)
    return reduced_embeddings

## 3. Pipeline Functions

Here we define the functions that encapsulate each major step of the pipeline, from loading data to evaluating the final results. This modular approach makes the process easy to follow and debug.

In [ ]:
def load_samples(path: PathLike) -> SampleCollection:
    """Loads samples from a CSV file."""
    logger.info(f"Loading samples from {path}...")
    samples = SampleCollection.from_csv(path)
    logger.info(f"Loaded {len(samples)} samples.")
    return samples

def process_samples(samples: SampleCollection) -> dict[CategoryType, SampleCollection]:
    """Groups samples by category and updates their embeddings and descriptions."""
    logger.info("Grouping samples by category and updating embeddings...")
    samples_by_category = samples.group_by_category()
    logger.info(f"Grouped samples into {len(samples_by_category)} categories.")

    logger.info("Describing samples in each category...")
    for category, sample_collection in tqdm(samples_by_category.items()):
        sample_collection.update_embeddings(feature_extractor)
        sample_collection.describe(describer)
    logger.info("Embeddings updated and samples described.")
    return samples_by_category

def create_instructions(samples: SampleCollection) -> InstructionCollection:
    """Creates and processes an InstructionCollection from a SampleCollection."""
    logger.info("Creating instruction collection from samples...")
    instructions = InstructionCollection.from_samples(samples)
    logger.info(f"Created an instruction collection with {len(instructions)} instructions.")

    logger.info("Updating instruction embeddings and clustering...")
    (
        instructions
        .update_embeddings(feature_extractor)
        .update_clusters(clustering_function=clustering_function, use_raw_instructions=False)
    )
    logger.info("Instruction embeddings updated and clusters created.")
    return instructions

def create_and_match_clusters(
        instructions: InstructionCollection,
        samples_by_category: dict[CategoryType, SampleCollection],
        all_samples: SampleCollection
) -> IdToCategoryResultType:
    """Describes instructions and matches them to sample categories."""
    logger.info("Grouping instructions by cluster...")
    instructions_by_cluster = instructions.group_by_cluster()
    logger.info(f"Grouped instructions into {len(instructions_by_cluster)} clusters.")

    logger.info("Describing instructions in each cluster...")
    for cluster, instruction_collection in tqdm(instructions_by_cluster.items()):
        instruction_collection.describe(describer)
    logger.info("Instructions described.")

    logger.info("Finding top similar sample categories for each instruction cluster...")
    id_to_category_pairs: IdToCategoryResultType = {}
    for cluster, instruction_collection in tqdm(instructions_by_cluster.items()):
        predicted_category = instruction_collection.get_cluster_category(
            sample_collections=list(samples_by_category.values()),
            similarity_function=similarity_function,
        )
        logger.info(f"Cluster N {instruction_collection.cluster} title: `{instruction_collection.title}` top similar sample category: {predicted_category}")
        logger.info(f"Mapping cluster {cluster} to category {predicted_category}")
        for sample in instruction_collection:
            id_to_category_pairs[sample.id] = (
                all_samples.get_sample_by_id(sample.id).category,
                predicted_category
            )
    logger.info("Matching completed.")
    logger.info(f"Total pairs: {len(id_to_category_pairs)}")
    return id_to_category_pairs

def store_results(cm: ConfusionMatrix, cluster_to_class_scores, storage_path: PathLike):
    """Saves the confusion matrix and scores to CSV files."""
    storage_path = Path(storage_path)
    os.makedirs(os.path.dirname(storage_path), exist_ok=True)
    logger.info(f"Storing evaluation results to {storage_path}...")
    cm.save_csv(storage_path / "confusion_matrix.csv")
    # Note: Storing cluster_to_class_scores would require converting the dict to a file format like json or csv.


## 4. Execute the Pipeline

This is the main execution block. It calls the pipeline functions in sequence and prints the final evaluation metrics. 

### Note on Subsetting Data:
To run the pipeline on a smaller portion of the data for faster testing, you can uncomment the line `samples: SampleCollection = samples[:1000]`.

In [ ]:
def main():
    """
    Main function to run the clustering pipeline.
    """
    start_time = time.time()

    # --- Step 1: Load Data ---
    samples = load_samples(CSV_PATH)
    
    # --- Optional: Uncomment to use a smaller subset of data for quick tests ---
    # samples: SampleCollection = samples[:1000]
    logger.info(f"Using {len(samples)} samples for processing.")

    # --- Step 2: Process Samples ---
    samples_by_category = process_samples(samples)
    
    # --- Step 3: Create Instructions and Cluster Them ---
    instructions = create_instructions(samples)
    
    # --- Step 4: Match Clusters to Categories ---
    id_to_category_pairs = create_and_match_clusters(
        instructions, samples_by_category, samples
    )

    # --- Step 5: Evaluate Results ---
    logger.info("Evaluating results...")
    cm = evaluate_results(id_to_category_pairs)
    
    # Prepare data for similarity score calculation
    predicted_cluster_list = []
    actual_category_list = []
    for id_, (actual_category, predicted_category) in id_to_category_pairs.items():
        predicted_cluster_list.append(predicted_category)
        actual_category_list.append(actual_category)
        
    # Calculate and print clustering quality scores
    cluster_to_class_scores = cluster_to_class_similarity_measures(
        predicted_cluster_list, actual_category_list
    )
    # Print the confusion matrix and detailed statistics
    logger.info("Evaluation results:")
    for measure, score in cluster_to_class_scores.items():
        print(f"{measure.capitalize()}: {score:.4f}")
    cm.print_matrix(sparse=True)
    cm.stat(summary=True)
        store_results(
        cm,
        cluster_to_class_scores,
        storage_path=Path(os.environ["EVALUATION_RESULTS_DIR"])
                     / f"results_{unique_folder_name}"
    )
    logger.info(f"Execution time: {time.time() - start_time:.2f} seconds")

# Run the main function
if __name__ == '__main__':
    main()